### Assignment

(A) Output: Number of rows and columns in an image

In [57]:
import cv2
import numpy as np
from imutils import *

## loading the image, applying a threshold
img = cv2.imread("images/image4.jpg",0)
(thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
img_bin = 255-img_bin
cv2.imwrite("images/image_bin1.jpg",img_bin)

## Here we are dividing the kernel length by 80, b/c we are counting rows& cols,
## we will increase it while counting number of cells, b/c the number of cells may decrease by col span one etc.
kernel_length = np.array(img).shape[1]//80
## defining vertical and horizontal kernel
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))

## erroding by vertical kernel, so that only vertical lines are left in the image
## dilating because the resulting lines are too thin
img_temp1 = cv2.erode(img_bin, vertical_kernel, iterations=2)
vertical_lines_img = cv2.dilate(img_temp1, vertical_kernel, iterations=3)
cv2.imwrite("images/vertical_lines.jpg", vertical_lines_img)

img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
cv2.imwrite("images/horizontal_lines.jpg",horizontal_lines_img)

## Applying contours function to horizontal lines to count total number of horizontal lines
im2, contoursx, hierarchy = cv2.findContours(horizontal_lines_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
im2, contoursy, hierarchy = cv2.findContours(vertical_lines_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

horizontal = []
horizontal1 = []


for a in range(len(contoursx)):
    for b in range(len(contoursx[a])):
        horizontal.append(contoursx[a][b][0])

for a in range(len(horizontal)):
    horizontal1.append(horizontal[a][1])
    
vertical = []
vertical1 = []

for a in range(len(contoursy)):
    for b in range(len(contoursy[a])):
        vertical.append(contoursy[a][b][0])

for a in range(len(vertical)):
    vertical1.append(vertical[a][0])

## In case of images with col span one, a line is counted twice, so to only take the unique values of y co-ord
Nrows = len(np.unique(horizontal1))-1
Ncols = len(np.unique(vertical1))-1
    
print("Number of rows : ",Nrows )
print("Number of columns : ",Ncols )

Number of rows :  4
Number of columns :  8


(B) Output: Number of cells in the grid

In [58]:
import cv2
import numpy as np
from imutils import *

img = cv2.imread("images/image4.jpg",0)
(thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
img_bin = 255-img_bin
cv2.imwrite("images/image_bin1.jpg",img_bin)

## kernel length decreased to capture things like col span one
kernel_length = np.array(img).shape[1]//320   
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

img_temp1 = cv2.erode(img_bin, vertical_kernel, iterations=1)
vertical_lines_img = cv2.dilate(img_temp1, vertical_kernel, iterations=3)
cv2.imwrite("images/vertical_lines.jpg", vertical_lines_img)

img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
cv2.imwrite("images/horizontal_lines.jpg",horizontal_lines_img)

## defining weighting factors
alpha = 0.5
beta = 1 - alpha

## adding both images, final bin image will be used for contour mapping
img_final_bin = cv2.addWeighted(vertical_lines_img, alpha, horizontal_lines_img, beta, 0.0)
img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY| cv2.THRESH_OTSU)
cv2.imwrite("images/img_final_bin.jpg",img_final_bin)

im2, contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

## drawing contours
img = cv2.drawContours(img, contours, -1, (0,255,0), 3)
cv2.imwrite("images/img.jpg",img)

Ncells = (len(contours)- 2)
print("Number of cells: ", Ncells)

Number of cells:  29


Hough Transform : To tackle rotated grid

The following cells contain incomplete work, will update you as soon as it is complete

An attempt to deal with rotated grid with **Hough line transform**

In [56]:

# import cv2
# import numpy as np

# img = cv2.imread('images/houghlines3.jpg')
# gray = cv2.imread('images/img_final_bin.jpg',0)
# # gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# edges = cv2.Canny(gray,50,150,apertureSize = 3)

# lines = cv2.HoughLines(edges,1,np.pi/180,200)
# # for rho,theta in lines[0]:
# #     a = np.cos(theta)
# #     b = np.sin(theta)
# #     x0 = a*rho
# #     y0 = b*rho
# #     x1 = int(x0 + 1000*(-b))
# #     y1 = int(y0 + 1000*(a))
# #     x2 = int(x0 - 1000*(-b))
# #     y2 = int(y0 - 1000*(a))

# #     cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

# # cv2.imwrite('houghlines3.jpg',img)

# print(edges)

None


An attempt to crop each cell so that we can latter perform optical character recognition

In [4]:
## (contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

## idx = 0
## for c in contours:
##     # Returns the location and width,height for every contour
##     x, y, w, h = cv2.boundingRect(c)
##     if (w > 80 and h > 20) and w > 3*h:
##         idx += 1
##         new_img = img[y:y+h, x:x+w]
##         cv2.imwrite(cropped_dir_path+str(idx) + '.png', new_img)

## # If the box height is greater then 20, widht is >80, then only save it as a box in "cropped/" folder.
##     if (w > 80 and h > 20) and w > 3*h:
##         idx += 1
##         new_img = img[y:y+h, x:x+w]
##         cv2.imwrite(cropped_dir_path+str(idx) + '.png', new_img)